In [ ]:
# fairing_fashion_mnist.ipynb
from kubeflow import fairing
from kubeflow.fairing.kubernetes.utils import mounting_pvc


def train():
    import datetime, os
    import tensorflow as tf

    mnist = tf.keras.datasets.fashion_mnist
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    print("x_train shape:", x_train.shape, "y_train shape:", y_train.shape)
    print("x_test shape:", x_test.shape, "y_test shape:", y_test.shape)

    x_train, x_test = x_train / 255.0, x_test / 255.0

    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='sgd',
                    loss='sparse_categorical_crossentropy',
                    metrics=['acc'])


    # 로컬 텐서보드 설정
    #log_dir = "log/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") 
    
    # fairing 텐서보드 설정
    #log_dir = "/notebook/log/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") 
    
    date_folder = datetime.datetime.now().strftime("%Y%m%d-%H%M%S") 
    
    # 조건문에 의한 로그 경로 설정 (로컬 실행인지, fairing에 의한 실행인지에 따라)
    if os.getenv('FAIRING_RUNTIME', None) is None:
        log_dir = "log/fit/" + date_folder
    else:
        log_dir = "/notebook/log/fit/" + date_folder   
    print(f"tensorboard log dir : {log_dir}")
    tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=log_dir,
                                                    histogram_freq=1)
    model.fit(x_train, y_train,
                verbose=1,
                validation_data=(x_test, y_test),
                epochs=10,
                callbacks=[tensorboard_cb])

In [ ]:

# USING DOCKERHUB
DOCKER_REGISTRY = 'YOURID' # 도커허브 아이디
# USING CAP HARBOR REGISTRY
# DOCKER_REGISTRY = 'cap.dudaji.com:31480/프로젝트 아이디'


# fairing 으로 함수 감싸기 1  
fairing.config.set_builder(
    'append',
    image_name='fairing-job',
    registry=DOCKER_REGISTRY,
    base_image='dudaji/cap-jupyterlab:tf2.0-cpu',
    push=True)


# mounting pvc 
notebook_volume = mounting_pvc(pvc_name="workspace-handson", 
                                pvc_mount_path="/notebook")

# fairing 으로 함수 감싸기 2
fairing.config.set_deployer('job',
                            pod_spec_mutators=[notebook_volume],
                            cleanup=True) # 잡을 실행후 완료시 잡을 삭제할지의 여부를 결정


if __name__ == '__main__':
    #print('local train()') # 로컬 실행시 사용
    #train() # 로컬 실행시 사용
    print('remote train()')
    remote_train = fairing.config.fn(train)
    remote_train()  
